 # Capstone Project: Chosing a location to build a Samsung supply chain in Abidjan

# Introduction & Background

The big firm Samsung plans to open some offices in Abidjan, Ivory Coast, in Africa. This project aim at winning the Ivorian phone market by building an effective supply chain in Abidjan which can reduce production cost, will supply the Ivorian phone Market and eventually the West African one with cheaper phones. One of the factors impacting the sales revenue is the location. That is why they contacted the best data scientist group from Ivory Coast called “The Pentagon” to carry out a study and pinpoint the best location. Abidjan is a city on the southern Atlantic coast of Ivory Coast, in West Africa. It is a capital of Ivory Coast and the major urban center with skyscrapers rising above the Ebrie Lagoon (Google). The country is well known for being a commercial hub for West Africa, with a GDP growth of 8% on average. Ivory Coast, West Africa do not have a firm making phone from West Africa. Therefore, winning Ivory Coast through Abidjan is winning the West African Market, and eventually Africa. The best location for the project Samsung involves leveraging the geographical data to solve that issue.

# Data & Methodology

The required data was divided into three groups:

Data for the list of neighborhoods in Abidjan-to be retrieved from Wikipedia.
The latitudes and longitudes for the city and the neighborhoods.
The data regarding the cities in Abidjan by Foursquare location data.
This information will be used with the K-Clustering method to establish cluster, and eventually a map.The features to take in consideration to find the best location are:
*the workforce availibily and labor skills generally located in the attractive industrial zone.
*presence of Airport, Railway Stations and Port which will be display by the folium map.
After analyzing ,this information, we will come up with the best location.

# Target Audience

The firm Samsung 

In [1]:
# import libraries
from bs4 import BeautifulSoup #use to scrape data online
import requests #use to scrape data online
import pandas as pd # read data into a dataframe format
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
from sklearn.cluster import KMeans # import k-means from clustering stage
import numpy as np
import matplotlib.cm as cm  # Matplotlib and associated plotting modules
import matplotlib.colors as colors
# use the inline backend to generate the plots within the browser
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

Matplotlib version:  3.1.3


# Get the coordinates of Abidjan city

In [2]:
# Grab the coordinates of Abidjan city

user_agent= "Abj_explorer"
address = 'ABIDJAN, ABJ'

geolocator = Nominatim(user_agent=user_agent)
location = geolocator.geocode(address)
latitude_abj = location.latitude
longitude_abj = location.longitude
print('The geograpical coordinates of Abidjan are {}, {}.'.format(latitude_abj, longitude_abj))

The geograpical coordinates of Abidjan are 5.264928299999999, -3.9259475068297522.


# Data import and processing

Little processing was needed since the data were fetched online using the following website https://www.latlong.net and then organised in an excel spreadsheet

In [3]:
# import the data set
import pandas as pd
path = r'C:\Users\kotero\Documents\COURSERA\Abidjan GPS cities Coordinates.xlsx'
df = pd.read_excel(path)
df

,States,Latitudes,Longitudes
0,Abobo,5.4329,-4.0389
1,Adjame,5.3651,-4.0236
2,Yopougon,5.3177,-4.0900
3,Plateau,5.3233,-4.0236
4,Attecoube,5.3362,-4.0414
5,Cocody,5.3602,-3.9674
6,Koumassi,5.3030,-3.9419
7,Marcory,5.3027,-3.9827
8,Port-Bouet,5.2773,-3.8859
9,Treichville,5.2921,-4.0134


In [4]:
# drop all the rows with NaN
df.dropna(how='all', axis=0, inplace=True)
df

,States,Latitudes,Longitudes
0,Abobo,5.4329,-4.0389
1,Adjame,5.3651,-4.0236
2,Yopougon,5.3177,-4.0900
3,Plateau,5.3233,-4.0236
4,Attecoube,5.3362,-4.0414
5,Cocody,5.3602,-3.9674
6,Koumassi,5.3030,-3.9419
7,Marcory,5.3027,-3.9827
8,Port-Bouet,5.2773,-3.8859
9,Treichville,5.2921,-4.0134


In [5]:
# create map of Abidjan using latitude and longitude values

map_abj = folium.Map(location=[latitude_abj, longitude_abj], zoom_start=11)

# add markers to map
for lat, lng, States in zip(df['Latitudes'],df['Longitudes'], df['States'],):
    label = '{}'.format(States)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_abj)  
    

# add pop-up text to each marker on the map
for lat, lng, label in zip(df['Latitudes'],df['Longitudes'],df['States'] ):
    folium.Marker([lat, lng], popup=label).add_to(map_abj)  
    
    
map_abj

 # Stamen Toner Map

Let's create a Stamen Toner map of Abidjan to explore river meanders and coastal zones. 

In [7]:
# create a Stamen Toner map of the world centered around Canada
Abidjan_map = folium.Map(location=[5.264928299999999, -3.9259475068297522], zoom_start=4, tiles='Stamen Toner')

# display map
Abidjan_map

# Next, let us start utilizing the Foursquare API to explore the neighborhoods and segment them

Define Foursquare Credentials and Version.

In [8]:
CLIENT_ID = 'UIRDM55V35DZKHRKFI1ZPK3NUTFWTAORAA3NPK2SOCV33BVM' # your Foursquare ID
CLIENT_SECRET = 'YQPSGYDERAPEYNORG2T4ATRQ4NSY1FCZT01A5SPJ3VWKVFP0' # your Foursquare Secret
VERSION = '20180604'

print('Your credentials:')
print('CLIENT_ID:' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID:UIRDM55V35DZKHRKFI1ZPK3NUTFWTAORAA3NPK2SOCV33BVM
CLIENT_SECRET:YQPSGYDERAPEYNORG2T4ATRQ4NSY1FCZT01A5SPJ3VWKVFP0


# Explore all the major venues around in and around the cities within a radius of 6km

In [9]:
LIMIT = 500 
radius = 6000 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=6000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Districts', 
                  'Districts Latitude', 
                  'Districts Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
abj_venues = getNearbyVenues(names=df['States'],
                                   latitudes=df['Latitudes'],
                                   longitudes=df['Longitudes']
                                  )

Abobo
Adjame
Yopougon
Plateau
Attecoube
Cocody
Koumassi
Marcory
Port-Bouet
Treichville
Ile Boulay


In [12]:
abj_venues.shape

(414, 7)

In [13]:
abj_venues.head(10)

,Districts,Districts Latitude,Districts Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abobo,5.4329,-4.0389,Ebimpe,5.451773,-4.073543,Train Station
1,Abobo,5.4329,-4.0389,Zoo,5.382706,-4.022097,Zoo
2,Adjame,5.3651,-4.0236,Norima,5.363668,-3.992067,American Restaurant
3,Adjame,5.3651,-4.0236,Bao Café,5.348778,-3.996881,Coffee Shop
4,Adjame,5.3651,-4.0236,le pub,5.375782,-3.998315,Bar
5,Adjame,5.3651,-4.0236,Des Gateaux & Du Pain,5.360270,-3.989671,Bakery
6,Adjame,5.3651,-4.0236,Casino Mandarine Danga,5.345053,-4.010700,Supermarket
7,Adjame,5.3651,-4.0236,Hollywood Boulevard,5.375848,-3.990464,Lounge
8,Adjame,5.3651,-4.0236,Sofitel Abidjan Hôtel Ivoire,5.327097,-4.004801,Hotel
9,Adjame,5.3651,-4.0236,Nuit De Saigon,5.371588,-3.990445,Vietnamese Restaurant


In [18]:
abj_venues['Venue Category'].value_counts()

Hotel                   40
Shopping Mall           26
African Restaurant      26
Nightclub               20
Fast Food Restaurant    19
                        ..
Beach                    1
Train Station            1
Duty-free Shop           1
Cafeteria                1
Shopping Plaza           1
Name: Venue Category, Length: 64, dtype: int64